In [1]:
## Activate the Packages
import yfinance as yf
import numpy as np
import pandas as pd

In [2]:
def trend_following_vs_buy_hold(tickers, period='6mo'):
    for ticker in tickers:
        # Download data
        data = yf.download(ticker, period=period, auto_adjust=False)
        if data.empty:
            print(f"{ticker} → No data available.\n")
            continue

        # Keep only Adj Close
        data = data[['Adj Close']].copy()

        # Drop rows with missing data
        data.dropna(inplace=True)

        # Moving Averages
        data['STMA12'] = data['Adj Close'].rolling(window=12).mean()
        data['LTMA26'] = data['Adj Close'].rolling(window=26).mean()

        # Daily Returns
        data['Daily Return'] = data['Adj Close'].pct_change()

        # Buy & Hold Return
        try:
            buy_price = data['Adj Close'].iloc[0]
            sell_price = data['Adj Close'].iloc[-1]
            buy_hold_return = (sell_price / buy_price - 1).iloc[0]
        except IndexError:
            buy_hold_return = 0.0

        # Trend Following Strategy
        data['TFSignal'] = (data['STMA12'] > data['LTMA26']).astype(int)
        tf_data = data[data['TFSignal'] == 1].copy()

        if not tf_data.empty:
            tf_data['TF Cumulative Return'] = (1 + tf_data['Daily Return']).cumprod() - 1
            tf_final_return = tf_data['TF Cumulative Return'].iloc[-1]
        else:
            tf_final_return = 0.0

        # Output Comparison
        print(f"\n📈 {ticker}")
        print(f"   Buy & Hold Return:         {buy_hold_return:.2%}")
        print(f"   Trend Following Return:    {tf_final_return:.2%}")



In [5]:
tickers = ['M&M.NS', 'ITC.NS', 'HINDUNILVR.NS', 'INFY.NS','TCS.NS']
trend_following_vs_buy_hold(tickers, period='1y')

[*********************100%***********************]  1 of 1 completed



📈 M&M.NS
   Buy & Hold Return:         11.91%
   Trend Following Return:    -22.15%


[*********************100%***********************]  1 of 1 completed



📈 ITC.NS
   Buy & Hold Return:         0.41%
   Trend Following Return:    -6.12%


[*********************100%***********************]  1 of 1 completed



📈 HINDUNILVR.NS
   Buy & Hold Return:         -6.53%
   Trend Following Return:    -5.85%


[*********************100%***********************]  1 of 1 completed



📈 INFY.NS
   Buy & Hold Return:         3.80%
   Trend Following Return:    13.09%


[*********************100%***********************]  1 of 1 completed


📈 TCS.NS
   Buy & Hold Return:         -10.64%
   Trend Following Return:    -5.16%
